In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from flask import Flask, jsonify, request

# Define a PyTorch dataset for the text data
class TextDataset(Dataset):
    def __init__(self, Dataset):
        self.data = pd.read_csv(Dataset)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = {'text': self.data.iloc[idx, :-1].values,
                  'label': self.data.iloc[idx, -1]}
        return sample

# Define a PyTorch model for the text data
class TextModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TextModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Load the text data into a PyTorch dataset
dataset = TextDataset('medical_data_Sheet1.csv')

# Create a PyTorch DataLoader for the text data
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

# Create a PyTorch model for the text data
model = TextModel(input_size=5, hidden_size=10, output_size=1)

# Define the loss function for the PyTorch model
criterion = nn.BCELoss()

# Define the optimizer for the PyTorch model
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the PyTorch model on the text data
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs = data['text']
        labels = data['label']
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(dataloader)))

# Define a Flask app for the PyTorch model
app = Flask(__name__)

# Define a Flask API endpoint for the PyTorch model
@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        data = request.get_json()
        input_data = torch.tensor(data['input_data'])
        output = model(input_data.float())
        return jsonify({'output': output.tolist()})

# Run the Flask app for the PyTorch model
if __name__ == '__main__':
    app.run(debug=True)

RuntimeError: DataLoader worker (pid(s) 15456, 644, 17936, 29092) exited unexpectedly